# AI Agent with LangSmith Tracing

<div>
<center><img src="./images/time_agent.png" width="600"/></center>
</div>

This exercise builds upon the first AI agent we created in **Notebook 1 - My First AI Agent**. We'll recreate the same ReAct AI agent using [**LangGraph**](https://www.langchain.com/langgraph), but this time we'll add **LangSmith tracing** to monitor and debug the agent's behavior in real-time.

The agent will have the same task: **providing the current date and time in a specific location**. It will respond to questions such as:

**"What time is it in Italy?"**

### What's New in This Exercise:

The key difference is the integration of **LangSmith tracing**, which allows you to:
- **Visualize the agent's execution flow**: See each step the agent takes, from receiving input to generating a response.
- **Debug tool calls**: Monitor which tools are invoked, with what parameters, and what they return.
- **Track performance metrics**: Measure latency, token usage, and other important metrics.
- **Analyze conversation history**: Review past interactions to understand agent behavior patterns.

LangSmith provides a powerful observability platform for LLM applications, making it easier to understand, debug, and optimize your AI agents.

Let's get started with setting up tracing!

## 0. Preparing the Environment and Setting Up LangSmith Tracing

Before building the AI agent, we need to set up our environment and import the necessary functions and libraries. In this exercise, we will use:

- [`load_dotenv`](https://pypi.org/project/python-dotenv/): To load environment variables (e.g., your OpenAI API key).
- [`ChatOpenAI`](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html): A language model interface for interacting with OpenAI's GPT models.
- [`create_react_agent`](https://langchain-ai.github.io/langgraph/reference/prebuilt/): A utility from **LangGraph** for building ReAct AI agents.
- [`tool`](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.convert.tool.html): A decorator to define tools that the agent can use during execution.
- `print_stream`: A utility to print the agent’s responses as they stream in.

### Setting Up LangSmith Tracing

To enable LangSmith tracing for your AI agent, you need to configure the following environment variables in your `.env` file:

**Required Environment Variables:**
- **`LANGSMITH_TRACING`**: Set to `"true"` to enable tracing.
- **`LANGSMITH_API_KEY`**: Your LangSmith API key (obtain it from [smith.langchain.com](https://smith.langchain.com)).
- **`OPENAI_API_KEY`**: Your OpenAI API key (or your LLM provider's API key).

**Optional Environment Variables:**
- **`LANGSMITH_WORKSPACE_ID`**: If your LangSmith account is linked to multiple workspaces, specify which workspace to use.
- **`LANGSMITH_PROJECT`**: The project name under which traces will be organized (defaults to "default" if not specified).

### Example `.env` Configuration:
```
LANGSMITH_TRACING=true
LANGSMITH_API_KEY=your_langsmith_api_key_here
OPENAI_API_KEY=your_openai_api_key_here
LANGSMITH_PROJECT=tech4tech-ai-agents
```

Once these variables are set, LangSmith will automatically capture and send tracing data every time your agent runs. You can view the traces in the LangSmith dashboard at [smith.langchain.com](https://smith.langchain.com).

**Note**: Make sure you have a LangSmith account. You can sign up for free at [smith.langchain.com](https://smith.langchain.com).

In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from utils.stream import print_stream

load_dotenv(override=True)

## 1. Initializing the Large Language Model (LLM)

The first step to create a new AI Agent with LangGraph is to initialize the OpenAI language model that our agent will use to process and respond to queries. When specifying the model, it's a good practice to:

1. **Specify the full model name**: To ensure compatibility and stability (e.g., `gpt-4o-mini-2024-07-18`).
2. **Start with a low temperature value**: A temperature of `0` ensures deterministic responses, making the agent focus on precision. You can increase the temperature later for more creative responses.

For this exercise, we will use the latest version of the `gpt-4o-mini` model (`gpt-4o-mini-2024-07-18`) with a temperature of `0`, but you can experiment with other models (the full list is available [here](https://platform.openai.com/docs/models#current-model-aliases)), temperatures, and other parameters to see how they affect the agent's behavior.

In [ ]:
MODEL_NAME = "openai:gpt-4o-mini-2024-07-18"
MODEL_TEMPERATURE = 0

llm = init_chat_model(model=MODEL_NAME, temperature=MODEL_TEMPERATURE)

You can learn more about ChatOpenAI in the official langchain documentation: https://python.langchain.com/docs/integrations/chat/openai/

## 2. Test the LLM

Before equipping our agent with tools, let’s verify that the underlying language model lacks access to real-time information such as the current date and time. This is expected behavior because the LLM operates on static knowledge and does not have tools or APIs to fetch live data on its own.

We’ll confirm this by asking the LLM the following question:  

**"What time is it in Italy?"**

Let’s run the code and observe the response.

In [ ]:
# You can learn more about langchain messages and roles in the official documentation: https://python.langchain.com/docs/concepts/messages/
messages = [
    ("user", "What time is it in Italy?")
]

response = llm.invoke(messages)

print("Assistant:", response.content)

## 3. Defining the Tools

Now that we have seen that the LLM cannot access real-time information on its own, we can give the AI agent a **tool** to fill this gap. Tools are external functions that the agent can use to assist in answering user queries.

### Tool Requirements:

For this exercise, we are building a tool to retrieve the current date and time for a given timezone. Here's what the tool should do:
1. Accept an input, specifically a timezone.
2. Determine the current date and time for that timezone.
3. Return the result to the agent.
4. Manage any errors that may occur during the process.

You can define your own implementation of the tool (recomended) or use the provided one.

To learn more about tools in LangGraph, you can refer to the official documentation: https://python.langchain.com/docs/concepts/tools/

In [ ]:
from tools.date_and_time import prebuilt_get_current_datetime

@tool
def get_current_datetime(timezone: str) -> str:
    """
    Get the current date and time in a specified timezone.

    Args:
        timezone (str): The timezone to get the current date and time for (e.g., "Europe/Rome").
    Returns:
        str: The current date and time in the specified timezone.
    """
    return prebuilt_get_current_datetime(timezone)

Test the tool by calling it with a sample timezone, such as `"Europe/Rome"`, and observe the output.

In [ ]:
get_current_datetime.invoke("Europe/Rome")

## 4. Crafting the Agent System Prompt

Before integrating tools and building our agent, we need to define the **system prompt**. The system prompt acts as the foundation for the agent's behavior by clearly outlining its role, purpose, instructions, and limitations.

Crafting the perfect system prompt is an iterative process, so you can start with a simple version and refine it as you test the agent's performance.

### Prompt Engineering Techniques:
Here are some techniques to keep in mind when defining the agent's system prompt:
1. **Role Prompting**: Clearly define the role of the agent (e.g., "You are D&T Bot, an agent specialized in providing the current date and time for different locations.").
2. [**Few-Shot Prompting**](https://www.promptingguide.ai/techniques/fewshot) (Optional): Provide a few examples of user questions and the appropriate responses to guide the agent further.

You can learn more about prompt engineering in the prompting engineering guide: [https://www.promptingguide.ai/](https://www.promptingguide.ai/)

### Your Task:
Define a system prompt for your agent. The agent should:
- Clearly convey its **role and purpose**.
- Provide the **current date and time** for a specific location when asked.
- Ask the user to specify a location if it is not provided.
- Politely decline to answer any queries unrelated to date and time.

### Things to Consider:
- **No Need to Describe Tools**: You do not need to specify the tools or their behaviors in the system prompt. The agent will already know how to use them based on the descriptions you provided earlier.
- **Be Creative**: You can inject some personality into your prompt if desired to give your agent a unique tone (e.g., friendly, formal, or professional).

In [ ]:
DATE_AND_TIME_AGENT_PROMPT = """
You are a helpful AI assistant that can provide the current date and time in various timezones. You have access to the following tool:
get_current_datetime: Get the current date and time in a specified timezone.

When you receive a question, determine if you need to use the tool to answer it. If you do, use the tool and provide the answer. If you don't need to use the tool, answer the question directly.

Tone: Friendly tone, use emojis, and be concise.
Response Format: Always respond in a single line, either with the tool's output or a direct answer.
"""

## 5. Building the AI Agent

Now that we have all the core components ready:
1. The **language model (LLM)** for processing queries.
2. A **tool** for retrieving the current date and time.
3. A well-crafted **system prompt** to guide the agent's behavior.

We’re ready to bring everything together into a fully functional AI agent. 

### How It Works:
We’ll use the **`create_react_agent`** function from LangGraph. This utility embeds the logic required to create a basic **ReAct (Reason + Act)** agent, allowing it to:
- Reason about the user’s query.
- Decide whether and how to use the provided tools.
- Generate meaningful responses, respecting the constraints of the system prompt.

### Key Parameters:
- **`model`**: The LLM we defined earlier.
- **`tools`**: A list of tools (e.g., the `get_current_datetime` tool).
- **`prompt`**: The system prompt crafted to guide the agent.

You can learn more about the `create_react_agent` function in the official LangGraph documentation: https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

graph = create_react_agent(
    model=llm, 
    tools=[get_current_datetime], 
    prompt=DATE_AND_TIME_AGENT_PROMPT, 
    checkpointer=memory)

The **ReAct agent** we just created isn't just a simple function; it's a structured graph that represents how the agent reasons, acts, and responds to user input. LangGraph allows us to visualize this graph, which helps us better understand:
- How the agent processes inputs.
- How its decision-making workflow is structured.

Let’s display the graph!

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

## 6. Testing the AI Agent

Now that the agent is fully built, it's time to test it! By sending a query to the agent, we can examine how it:
1. Processes the input question.
2. Reasonably decides if and how to use tools like `get_current_datetime`.
3. Produces an accurate and user-friendly response.

### Testing Query:
Let’s try the same query we used to test the LLM:

**"What time is it in Italy?"**

To verify that the agent is now capable of providing the current date and time for a specific location.

In [ ]:
from uuid import uuid4

question = """What time is it in Italy?"""
inputs = {"messages": [("user", question)]}

config = {
    "configurable": {
        "thread_id": str(uuid4())
        }
}

print_stream(graph.stream(inputs, config, stream_mode="values"))

Let's try a more complex query:

**"What time is it in Italy and California?"**

This query is intentionally a little tricky so we can observe how the agent handles both locations and whether it uses tools appropriately.

In [ ]:
question = """What time is it in Italy and California?"""

inputs = {"messages": [("user", question)]}

print_stream(graph.stream(inputs, config, stream_mode="values"))

### Iterative Testing:

Now that your agent is up and running with LangSmith tracing enabled, it's time to test it with your own queries! This hands-on testing allows you to evaluate:
1. **How well the agent responds to expected queries** (e.g., asking the time in different locations).
2. **Whether it handles unexpected or ambiguous queries gracefully** (e.g., if no location is provided or irrelevant questions are asked).
3. **How effectively it uses the tools** to fulfill its purpose.

### Monitoring with LangSmith:

While testing, **open your LangSmith dashboard** at [smith.langchain.com](https://smith.langchain.com) to observe:
- **Real-time traces**: Watch each agent execution as it happens.
- **Tool invocations**: See which tools are called, with what parameters, and what they return.
- **Token usage and costs**: Monitor how many tokens each interaction consumes.
- **Latency metrics**: Identify performance bottlenecks in your agent's workflow.
- **Error tracking**: Quickly spot and debug any failures or unexpected behaviors.

### Iterative Process:

Crafting the perfect agent is an **iterative process**. Use these trials to identify areas of improvement and adjust the agent as needed. Consider modifying the following aspects:
- **System Prompt**: Adjust the wording to clarify the agent's purpose, add reasoning steps, or inject personality.
- **Tool Definition**: Rename tools, change their parameters, or improve their descriptions for clarity and better integration.
- **Model Settings**: Experiment with model temperature to balance determinism (`temperature=0`) and creativity (higher values).
- **Tool Limitations**: For example, add error handling if an invalid timezone is passed.

### Tips for Effective Refinement:
- Use **LangSmith traces** to review the agent's reasoning steps and identify where improvements are needed.
- Compare multiple runs in LangSmith to understand how changes affect behavior.
- Test with a wider variety of queries to account for edge cases.
- Iterate gradually, making small adjustments to prompts, tools, or configurations one step at a time.

Remember, building an AI agent is like sculpting—it evolves over time with refinement and experimentation. LangSmith makes this process more transparent and efficient!

In [ ]:
question = """<YOUR QUESTION HERE>"""

inputs = {"messages": [("user", question)]}

print_stream(graph.stream(inputs, config, stream_mode="values"))

## 7. Conclusion

🎉 Congratulations on Completing the Exercise! 🎉

Well done! You've successfully built your first AI agent with **LangSmith tracing** enabled. This exercise demonstrated how to:

### What We've Accomplished:
1. **Set up LangSmith tracing**: You configured the necessary environment variables to enable real-time monitoring of your AI agent.
2. **Built a traced AI agent**: You recreated the date and time agent from Notebook 1, but now with full observability through LangSmith.
3. **Monitored agent execution**: You learned how to use the LangSmith dashboard to observe traces, tool calls, token usage, and performance metrics.
4. **Debugged with visibility**: You saw how tracing makes it easier to understand what your agent is doing at each step and identify areas for improvement.

### The Power of LangSmith Tracing:

LangSmith transforms the development experience by providing:
- **Complete visibility** into every step of your agent's execution
- **Performance insights** to optimize speed and cost
- **Debugging capabilities** to quickly identify and fix issues
- **Collaboration tools** to share traces with team members
- **Production monitoring** for deployed applications

### What's Next?

You're now ready to apply LangSmith tracing to more complex agents! In the **next notebook (6 - Multi-agent with tracing.ipynb)**, you'll explore how to:
- Trace multi-agent systems with multiple interacting agents
- Monitor complex workflows with multiple tools execution
- Debug coordination between different agents

LangSmith becomes even more valuable as your agents grow in complexity, making it an essential tool for production AI applications.

🚀 Continue to the next notebook to see LangSmith in action with multi-agent systems!